In [1]:
import pandas as pd
from glob import glob
import os

from helpers_process_raw_data import load_waveform, load_lfp_data, expand_data_to_single_trials

In [3]:
data = pd.DataFrame()
for folder in glob("raw/*", recursive=True):
    if os.path.isdir(folder):
        print(folder)
        fnames = glob(f"{folder}/*lfp_means.mat")
        waveforms_fname = glob(f"{folder}/waveform*.mat")
        if len(waveforms_fname) == 1:
            fish_id = folder.split("/")[-1].split("-")[0]
            waveforms_fname = waveforms_fname[0]
            waveforms = load_waveform(waveforms_fname)
            new_lfp_data = pd.DataFrame()
            for fname in fnames:
                lfp_data = load_lfp_data(fname)
                new_lfp_data = pd.concat([new_lfp_data, lfp_data], axis=0, ignore_index=True)
            new_lfp_data["fish_id"] = fish_id
            new_lfp_data["paired_experiment"] = "paired" in folder
            new_lfp_data = pd.merge(new_lfp_data, waveforms, on="stimulus_marker")
        else:
            print(f"Folder {folder} does not contain a waveform file or contains more than one waveform file.")
            continue
        data = pd.concat([data, new_lfp_data], axis=0, ignore_index=True)

data_single_trials = data.apply(expand_data_to_single_trials, axis=1)  # type: ignore
data_single_trials = pd.concat(data_single_trials.tolist(), axis=0, ignore_index=True)
data.to_pickle("processed/trial_averages.pkl")
data_single_trials.to_pickle("processed/single_trials.pkl")

raw/fish_03-20190711-separate
raw/fish_12-20200826-paired
raw/fish_08-20200714-mz
raw/fish_02-20190617-dlz
raw/fish_06-20200623-separate
raw/fish_13-20200902-paired
raw/fish_01-20190605-separate
raw/fish_10-20200722-separate
raw/fish_07-20200626-dlz
raw/fish_04-20190731_dlz
raw/fish_09-20200715-separate
raw/fish_11-20200806-paired
raw/fish_05-20190910-mz
